### Test trivial case, to see if I can access Tiingo's Restful API

### Tiingo Documentation:
- Data rate limits for free tier: https://www.tiingo.com/about/pricing
- How to get end-of-day prices: https://www.tiingo.com/documentation/end-of-day

In [1]:
# Standard library imports
import sys
import os

import pandas as pd
import numpy as np

from datetime import datetime, date
from typing import List, Set, Dict, Any

from dotenv import load_dotenv
import requests

# /home/ubuntu/financial-etl-poc/this_folder
current_folder = os.getcwd()

# /home/ubuntu/financial-etl-poc/
project_root_folder = os.path.abspath(os.path.join(current_folder, ".."))
sys.path.append(project_root_folder)

# For loading credentials from .env under financial-etl-poc
dotenv_path = os.path.join(project_root_folder, ".env")
load_dotenv(dotenv_path)

True

In [2]:
tiingo_api_token = os.getenv("tiingo_api_token")
tiingo_url = f"https://api.tiingo.com/api/test?token={tiingo_api_token}"

headers = {'Content-Type': 'application/json'}
requestResponse = requests.get(tiingo_url, headers = headers)
requestResponse.json()

{'message': 'You successfully sent a request'}

### Test non-trivial case for one stock and a few dates

In [3]:
# Test to see how Tiingo handles dividends and stock splits
# Nvidia was $120.89 on 2024-06-07 after adjustment. This was before a 10:1 stock split and a few dividend payments

start_date = "2024-06-07"
end_date = "2024-06-07"
ticker = "NVDA"
freq = "daily"
format = "json"

url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices?startDate={start_date}&endDate={end_date}&resampleFreq={freq}&format={format}&token={tiingo_api_token}"

params = {
    "startDate": start_date,
    "endDate": end_date,
    "format": "csv",
    "resampleFreq": "daily"
}

headers = {'Content-Type': 'application/json'}

response = requests.get(url, headers=headers)

In [4]:
# Returns a list of dictionaries
response.json()

[{'date': '2024-06-07T00:00:00.000Z',
  'close': 1208.88,
  'high': 1216.9171,
  'low': 1180.22,
  'open': 1197.7,
  'volume': 41238580,
  'adjClose': 120.849084499,
  'adjHigh': 121.6525357738,
  'adjLow': 117.9840071036,
  'adjOpen': 119.7314443985,
  'adjVolume': 412385800,
  'divCash': 0.0,
  'splitFactor': 1.0}]

In [5]:
# Cleaner version
start_date = "2024-06-07"
end_date = "2024-06-12"
ticker = "NVDA"

url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices"

params = {
    "startDate": start_date,
    "endDate": end_date,
    "resampleFreq": "daily",
    "format": "json",
    "token": tiingo_api_token
}

headers = {'Content-Type': 'application/json'}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()  # always good for catching HTTP errors

In [6]:
# Returns a list of dictionaries
response.json()[0:2]

[{'date': '2024-06-07T00:00:00.000Z',
  'close': 1208.88,
  'high': 1216.9171,
  'low': 1180.22,
  'open': 1197.7,
  'volume': 41238580,
  'adjClose': 120.849084499,
  'adjHigh': 121.6525357738,
  'adjLow': 117.9840071036,
  'adjOpen': 119.7314443985,
  'adjVolume': 412385800,
  'divCash': 0.0,
  'splitFactor': 1.0},
 {'date': '2024-06-10T00:00:00.000Z',
  'close': 121.79,
  'high': 123.1,
  'low': 117.01,
  'open': 120.37,
  'volume': 308134791,
  'adjClose': 121.7507941329,
  'adjHigh': 123.060372426,
  'adjLow': 116.9723328803,
  'adjOpen': 120.3312512503,
  'adjVolume': 308134791,
  'divCash': 0.0,
  'splitFactor': 10.0}]

### Function to pull closing price data for a single stock on a single date

In [7]:
def download_tiingo_closing_prices(business_date: date, ticker: str) -> dict:
    """
    Downloads closing prices from Tiingo for one date and one ticker, and returns result as dict
    """

    url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices"

    headers = {'Content-Type': 'application/json'}

    params = {
        "startDate": business_date,
        "endDate": business_date,
        "resampleFreq": "daily",
        "format": "json",
        "token": tiingo_api_token
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  # always good for catching HTTP errors
    return response.json()[0]

In [8]:
# Test
business_date = datetime(year = 2025, month = 5, day = 20)
ticker =  "AAPL"
dict_one_date_and_one_ticker = download_tiingo_closing_prices(business_date, ticker)
dict_one_date_and_one_ticker

{'date': '2025-05-20T00:00:00.000Z',
 'close': 206.86,
 'high': 208.47,
 'low': 205.03,
 'open': 207.67,
 'volume': 42496635,
 'adjClose': 206.86,
 'adjHigh': 208.47,
 'adjLow': 205.03,
 'adjOpen': 207.67,
 'adjVolume': 42496635,
 'divCash': 0.0,
 'splitFactor': 1.0}

### Function to pull closing price data for multiple stocks over multiple dates

In [9]:
def download_tiingo_closing_prices(start_date: date, end_date: date, tickers: List[str]) -> pd.DataFrame:
    """
    Downloads closing prices from Tiingo between start_date and end_date for list of tickers, and returns result as DataFrame
    """

    dfs = []

    for ticker in tickers:
        url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices"

        headers = {'Content-Type': 'application/json'}

        params = {
            "startDate": start_date,
            "endDate": end_date,
            "resampleFreq": "daily",
            "format": "json",
            "token": tiingo_api_token
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # always good for catching HTTP errors

        # response.json is a list of dict, where each dict is price/volume/ratio data for one business_date
        df_payloads_for_one_ticker_between_start_end_dates = pd.DataFrame(response.json())
        df_payloads_for_one_ticker_between_start_end_dates["ticker"] = ticker

        dfs.append(df_payloads_for_one_ticker_between_start_end_dates)
    
    df = pd.concat(dfs, axis = 0)

    return df
    # output dataframe should have columns (ticker, business_date, raw_json_payload/dict)

In [10]:
# Test
start_date = datetime(year = 2025, month = 5, day = 16)
end_date = datetime(year = 2025, month = 5, day = 19)
tickers =  ["AAPL", "SPY"]
download_tiingo_closing_prices(start_date, end_date, tickers)

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,ticker
0,2025-05-16T00:00:00.000Z,211.26,212.57,209.77,212.36,54737850,211.26,212.57,209.77,212.36,54737850,0.0,1.0,AAPL
1,2025-05-19T00:00:00.000Z,208.78,209.48,204.26,207.91,46140527,208.78,209.48,204.26,207.91,46140527,0.0,1.0,AAPL
0,2025-05-16T00:00:00.000Z,594.20,594.50,589.28,591.25,76052101,594.20,594.50,589.28,591.25,76052101,0.0,1.0,SPY
1,2025-05-19T00:00:00.000Z,594.85,595.54,588.10,588.10,68168509,594.85,595.54,588.10,588.10,68168509,0.0,1.0,SPY


In [ ]:
# Test for handling exceptions, i.e. - ticker is invalid
# response.raise_for_status() raises "HTTPError: 4040 Client Error"
start_date = datetime(year = 2025, month = 5, day = 16)
end_date = datetime(year = 2025, month = 5, day = 19)
tickers =  ["BLAHBLAH"]
download_tiingo_closing_prices(start_date, end_date, tickers)

HTTPError: 404 Client Error: Not Found for url: https://api.tiingo.com/tiingo/daily/BLAHBLAH/prices?startDate=2025-05-16+00%3A00%3A00&endDate=2025-05-19+00%3A00%3A00&resampleFreq=daily&format=json&token=8f6adae2ff90b64814317d38d67878446f95e902